# Custom Text Classification Solution

## Load Azure Configurations

In [56]:
import os

# Load Azure configurations from environment variables
# Ensure that AZURE_AI_LANGUAGE_KEY and AZURE_AI_LANGUAGE_ENDPOINT are set in your environment
language_key = os.environ.get('AZURE_AI_LANGUAGE_KEY')
language_endpoint = os.environ.get('AZURE_AI_LANGUAGE_ENDPOINT')

project_name = "EmailClassifier"
model_name = "emailclassifiermodel"

## Import Project Job

In [58]:
import json

# Path to the JSON file
file_path = "emailLabel.json"

# Load the JSON file
with open(file_path, "r") as file:
    json_data = json.load(file)


In [75]:
import requests
import json

# Create a new analyzer
def import_project(request_body):
    
    url = f"{language_endpoint}/language/authoring/analyze-text/projects/{project_name}/:import?api-version=2022-05-01"
    headers = {
        "Ocp-Apim-Subscription-Key": language_key,
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(request_body))
    # The 201 (Created) response includes an Operation-Location header containing a URL that you can use to track the status of this asynchronous creation operation.
    operation_location = response.headers.get("Operation-Location")
    return operation_location

import_result = import_project(json_data)
print(f"Import result: {import_result}")

Import result: https://ziggylanguagedemocomplete.cognitiveservices.azure.com/language/authoring/analyze-text/projects/EmailClassifier/import/jobs/2df461a9-f628-4c51-b01c-f8fb17a154d5_638803584000000000?api-version=2022-05-01


## Get Import Job Status

In [76]:
def import_project_status(import_result):
    
    headers = {
        "Ocp-Apim-Subscription-Key": language_key,
    }
    
    response = requests.get(import_result, headers=headers)

    if response.status_code == 200:  # Success
        # Parse the JSON response
        response_data = response.json()
        # Pretty print the JSON data with indentation
        print(json.dumps(response_data, indent=4))
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [77]:
import_project_status(import_result)

{
    "jobId": "2df461a9-f628-4c51-b01c-f8fb17a154d5_638803584000000000",
    "createdDateTime": "2025-04-16T18:14:03Z",
    "lastUpdatedDateTime": "2025-04-16T18:14:03Z",
    "expirationDateTime": "2025-04-23T18:14:03Z",
    "status": "succeeded"
}


## Train your model

In [78]:
train_body = {
	"modelLabel": model_name,
	"trainingConfigVersion": "2022-05-01",
	"evaluationOptions": {
		"kind": "percentage",
		"trainingSplitPercentage": 80,
		"testingSplitPercentage": 20
	}
}

In [79]:
import requests
import json

# Create a new analyzer
def train_model(request_body):
    
    url = f"{language_endpoint}/language/authoring/analyze-text/projects/{project_name}/:train?api-version=2022-05-01"

    headers = {
        "Ocp-Apim-Subscription-Key": language_key,
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(request_body))
    # The 201 (Created) response includes an Operation-Location header containing a URL that you can use to track the status of this asynchronous creation operation.
    operation_location = response.headers.get("Operation-Location")
    return operation_location

In [80]:
train_result = train_model(train_body)
print(f"Train result: {train_result}")

Train result: https://ziggylanguagedemocomplete.cognitiveservices.azure.com/language/authoring/analyze-text/projects/EmailClassifier/train/jobs/5fad969b-7189-4c76-9dfe-6379c080431a_638803584000000000?api-version=2022-05-01


## Get training job status

In [81]:
def train_model_status(train_result):
    
    headers = {
        "Ocp-Apim-Subscription-Key": language_key,
    }
        
    response = requests.get(train_result, headers=headers)
        
    if response.status_code == 200:  # Success
        # Parse the JSON response
        response_data = response.json()
        # Pretty print the JSON data with indentation
        print(json.dumps(response_data, indent=4))
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [93]:
train_model_status(train_result)

{
    "result": {
        "modelLabel": "emailclassifiermodel",
        "trainingConfigVersion": "2022-05-01",
        "trainingStatus": {
            "percentComplete": 100,
            "startDateTime": "2025-04-16T18:14:18.7987661Z",
            "endDateTime": "2025-04-16T18:14:24.5313026Z",
            "status": "succeeded"
        },
        "evaluationStatus": {
            "percentComplete": 100,
            "startDateTime": "2025-04-16T18:14:25.5792561Z",
            "endDateTime": "2025-04-16T18:14:42.9470222Z",
            "status": "succeeded"
        }
    },
    "jobId": "5fad969b-7189-4c76-9dfe-6379c080431a_638803584000000000",
    "createdDateTime": "2025-04-16T18:14:17Z",
    "lastUpdatedDateTime": "2025-04-16T18:15:26Z",
    "expirationDateTime": "2025-04-23T18:14:17Z",
    "status": "succeeded",
    "warnings": []
}


## View your text classification model's evaluation and details

In [94]:
def view_model_details():
    
    url = f"{language_endpoint}/language/authoring/analyze-text/projects/{project_name}/models/{model_name}/evaluation/summary-result?api-version=2022-05-01"

    headers = {
        "Ocp-Apim-Subscription-Key": language_key,
    }
        
    response = requests.get(url, headers=headers)
        
    if response.status_code == 200:  # Success
        # Parse the JSON response
        response_data = response.json()
        # Pretty print the JSON data with indentation
        print(json.dumps(response_data, indent=4))
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [95]:
view_model_details()

{
    "projectKind": "CustomSingleLabelClassification",
    "customSingleLabelClassificationEvaluation": {
        "confusionMatrix": {
            "Account_Management": {
                "Account_Management": {
                    "normalizedValue": 40.0,
                    "rawValue": 2.0
                },
                "Billing_and_Payments": {
                    "normalizedValue": 60.0,
                    "rawValue": 3.0
                }
            },
            "Billing_and_Payments": {
                "Billing_and_Payments": {
                    "normalizedValue": 100.0,
                    "rawValue": 5.0
                }
            },
            "Order_and_Shipping": {
                "Order_and_Shipping": {
                    "normalizedValue": 100.0,
                    "rawValue": 5.0
                }
            },
            "Product_Information": {
                "Product_Information": {
                    "normalizedValue": 100.0,
                    "r

## Deploy your model

In [96]:
def deploy_model():
    
    url = f"{language_endpoint}/language/authoring/analyze-text/projects/{project_name}/deployments/{model_name}?api-version=2022-05-01"

    body = {
        "trainedModelLabel": model_name
    }

    headers = {
        "Ocp-Apim-Subscription-Key": language_key,
    }
        
    response = requests.put(url, headers=headers, data=json.dumps(body))
        
    # The 201 (Created) response includes an Operation-Location header containing a URL that you can use to track the status of this asynchronous creation operation.
    operation_location = response.headers.get("Operation-Location")
    return operation_location

In [97]:
deploy_result = deploy_model()
print(f"Deploy result: {deploy_result}")

Deploy result: https://ziggylanguagedemocomplete.cognitiveservices.azure.com/language/authoring/analyze-text/projects/EmailClassifier/deployments/emailclassifiermodel/jobs/c46929b7-270b-467d-8bd0-1856ce842f77_638803584000000000?api-version=2022-05-01


## Get deployment job status

In [98]:
def view_deployment_status(deploy_result):
    
    
    headers = {
        "Ocp-Apim-Subscription-Key": language_key,
    }
        
    response = requests.get(deploy_result, headers=headers)
        
    if response.status_code == 200:  # Success
        # Parse the JSON response
        response_data = response.json()
        # Pretty print the JSON data with indentation
        print(json.dumps(response_data, indent=4))
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [99]:
view_deployment_status(deploy_result)

{
    "jobId": "c46929b7-270b-467d-8bd0-1856ce842f77_638803584000000000",
    "createdDateTime": "2025-04-16T18:16:21Z",
    "lastUpdatedDateTime": "2025-04-16T18:16:24Z",
    "expirationDateTime": "2025-04-23T18:16:21Z",
    "status": "succeeded"
}


## Perform Classification

In [100]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using Azure Key and Endpoint
def authenticate_client():
    """
    Authenticates the Azure Text Analytics client using the provided key and endpoint.

    Returns:
        TextAnalyticsClient: An authenticated client for Azure Text Analytics.
    """
    ta_credential = AzureKeyCredential(language_key)
    text_analytics_client = TextAnalyticsClient(
        endpoint=language_endpoint,
        credential=ta_credential
    )
    return text_analytics_client

# Initialize the client
text_analytics_client = authenticate_client()

In [101]:
document = [
    """
        Subject: Unable to connect to the internet

        Hi Support Team,

        I am facing issues with my internet connection. It keeps disconnecting every few minutes. I have tried restarting my router but the problem persists. Please help me resolve this issue.

        Thanks,
        John Doe
    """,
    """
        Subject: Incorrect charge on my account

        Hello,

        I noticed an incorrect charge on my account for the month of March. I was charged twice for the same service. Can you please look into this and issue a refund?

        Regards,
        Jane Smith
    """,
    """
        Subject: Password reset request

        Dear Support,

        I am unable to log into my account as I forgot my password. Can you assist me with resetting my password?

        Thank you,
        Michael Johnson
    """,
    """
        Subject: Inquiry about product features

        Hi,

        I am interested in purchasing your new smartphone model. Can you provide more details about its features, especially the camera specifications?

        Best,
        Emily Davis
    """,
    """ 
        Subject: Order not received

        Hello,

        I placed an order two weeks ago and have not yet received it. The tracking number shows that it is still in transit. Can you provide an update on the delivery status?

        Thanks,
        Chris Brown
    """

]

In [106]:
def single_label_classify(document):
    """
    Classifies the given document into multiple labels using the Azure Text Analytics client.

    Args:
        document (list): A list of documents to classify.

    Returns:
        list: A list of classification results for each document.
    """
    # Start the multi-label classification process for the given document
    poller = text_analytics_client.begin_single_label_classify(
        document,
        project_name=project_name,
        deployment_name=model_name
    )

    # Retrieve the classification results once the operation is complete
    document_results = poller.result()

    # Iterate through the documents and their corresponding classification results
    # Iterate through the documents and their corresponding classification results
    for doc, classification_result in zip(document, document_results):
        # Check if the classification result is of type CustomDocumentClassification
        if classification_result.kind == "CustomDocumentClassification":
            # Retrieve the first classification result
            classification = classification_result.classifications[0]
            # Print the classification details including category and confidence score
            print("The document text '{}' was classified as '{}' with confidence score {}.".format(
                doc, classification.category, classification.confidence_score)
            )
            print()
        # Handle cases where the classification result contains an error
        elif classification_result.is_error is True:
            # Print the error details including code and message
            print("Document text '{}' has an error with code '{}' and message '{}'".format(
                doc, classification_result.error.code, classification_result.error.message
            ))
            

In [107]:
single_label_classify(document)

The document text '
        Subject: Unable to connect to the internet

        Hi Support Team,

        I am facing issues with my internet connection. It keeps disconnecting every few minutes. I have tried restarting my router but the problem persists. Please help me resolve this issue.

        Thanks,
        John Doe
    ' was classified as 'Technical_Support' with confidence score 0.23.

The document text '
        Subject: Incorrect charge on my account

        Hello,

        I noticed an incorrect charge on my account for the month of March. I was charged twice for the same service. Can you please look into this and issue a refund?

        Regards,
        Jane Smith
    ' was classified as 'Billing_and_Payments' with confidence score 0.24.

The document text '
        Subject: Password reset request

        Dear Support,

        I am unable to log into my account as I forgot my password. Can you assist me with resetting my password?

        Thank you,
        Michael Jo